In [5]:
!pip install pydub
!pip install pytube

In [0]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2 
import os
import sys
import moviepy.editor as mp


from math import ceil
from math import floor

from pydub import AudioSegment
from pytube import YouTube

import shutil

import IPython.display as ipd
import librosa
import librosa.display

import dlib

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
PATH = './drive/My Drive/TIP/Dataset/'
PATH_TO_MAIN = './drive/My Drive/TIP/'
YT_LINK = 'www.youtube.com/watch?v='

In [0]:
column_names = ['YouTube_ID', 'start_segment', 'end_segment', 'X_coordinate', 'Y_coordinate']
train_df = pd.read_csv(PATH + 'avspeech_train.csv',names = column_names)
test_df = pd.read_csv(PATH + 'avspeech_test.csv',names = column_names)

In [10]:
train_df.head()

,YouTube_ID,start_segment,end_segment,X_coordinate,Y_coordinate
0,CJoOwXcjhds,233.266000,239.367000,0.780469,0.670833
1,AvWWVOgaMlk,90.000000,93.566667,0.586719,0.311111
2,Y8HMIm8mdns,171.607767,174.607767,0.505729,0.240741
3,akwvpAiLFk0,144.680000,150.000000,0.698438,0.288889
4,Swss72CHSWg,90.023267,97.297200,0.230729,0.204630


In [11]:
test_df.head()

,YouTube_ID,start_segment,end_segment,X_coordinate,Y_coordinate
0,u5MPyrRJPmc,108.2400,111.240000,0.849219,0.305556
1,H1ulMfj5wRY,112.3200,116.940000,0.112500,0.345833
2,-wuxbgMRIWs,30.0300,36.745044,0.744531,0.211111
3,GNRPRH-E-sI,30.2302,38.171467,0.333594,0.494444
4,VvcwAGkSy2o,240.2000,253.366667,0.491667,0.372222


In [12]:
YT_LINK + train_df.iloc[2]["YouTube_ID"]

'www.youtube.com/watch?v=Y8HMIm8mdns'

In [34]:

shutil.rmtree('Images')
shutil.rmtree('Audio')
shutil.rmtree('Videos')
os.mkdir("Images")
os.mkdir("Audio")
os.mkdir("Videos")
len_train = train_df.shape[0]
count_unsuccessful = 0

for i in range(1,3):

  print(YT_LINK + train_df.iloc[i]["YouTube_ID"])
  try:
    yt = YouTube(YT_LINK + train_df.iloc[i]["YouTube_ID"])
    stream = yt.streams.first()
    stream.download(output_path = 'Videos/',filename = train_df.iloc[i]["YouTube_ID"])


    cap = cv2.VideoCapture("Videos/" + train_df.iloc[i]["YouTube_ID"] + '.mp4') 
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    count = 0

    dir_name = train_df.iloc[i]["YouTube_ID"]
    # os.mkdir("Images/" + dir_name)
    
    print("FPS is : "  + str(fps))
    start_cnt = train_df.iloc[i]["start_segment"]
    end_cnt = train_df.iloc[i]["end_segment"]
    
    while(1):

        ret,frame = cap.read()

        if ret == 1:

          count += 1

          (h, w) = frame.shape[:2]


          # if count > ceil(ceil(start_cnt)*fps) and count < floor(floor(end_cnt)*fps):
          #   cv2.imwrite("Images/" + dir_name + "/frame%d.png" % count, frame)

          if (((start_cnt + end_cnt)*fps)/2) - 16 < count < (((start_cnt + end_cnt)*fps)/2) + 16:
            dnnFaceDetector = dlib.cnn_face_detection_model_v1( PATH_TO_MAIN + "Face_detection/mmod_human_face_detector.dat")
            faceRects = dnnFaceDetector(frame, 0)
            print("Here hahaha")
            print(faceRects)
            for faceRect in faceRects:
                x1 = faceRect.rect.left()
                y1 = faceRect.rect.top()
                x2 = faceRect.rect.right()
                y2 = faceRect.rect.bottom()
                img = frame[y1:y2, x1:x2]
                print("About to crop")
                print(count)
                img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
                cv2.imwrite('Images/' +dir_name + '.png', img)
            break


          if count > int(end_cnt*fps):
            break
          k = cv2.waitKey(1)
          if k == ord('q'):
            break



    cap.release()
    cv2.destroyAllWindows()
  except:
    print("SORRY!!! The video is unavailable")
    count_unsuccessful += 1
    
    try:
      clip = mp.VideoFileClip("Videos/" + train_df.iloc[i]["YouTube_ID"] + '.mp4').subclip(start_cnt,end_cnt)
      clip.audio.write_audiofile("Audio/" + train_df.iloc[i]["YouTube_ID"] +'.mp3') 
    except:
      print("Error with audio")  

www.youtube.com/watch?v=AvWWVOgaMlk
FPS is : 30.000015705169986
Here hahaha
mmod_rectangles[[(532, 80) (871, 420)]]
About to crop
2738
[MoviePy] Writing audio in Audio/AvWWVOgaMlk.mp3


100%|██████████| 79/79 [00:00<00:00, 520.57it/s]

[MoviePy] Done.


www.youtube.com/watch?v=Y8HMIm8mdns
FPS is : 29.97002997002997
Here hahaha
mmod_rectangles[[(574, 114) (710, 251)]]
About to crop
5173
[MoviePy] Writing audio in Audio/Y8HMIm8mdns.mp3


100%|██████████| 67/67 [00:00<00:00, 512.00it/s]

[MoviePy] Done.
www.youtube.com/watch?v=akwvpAiLFk0


FPS is : 25.0
Here hahaha
mmod_rectangles[[(822, 150) (986, 314)]]
About to crop
3668
[MoviePy] Writing audio in Audio/akwvpAiLFk0.mp3


100%|██████████| 118/118 [00:00<00:00, 487.88it/s]

[MoviePy] Done.


In [0]:
for filename in os.listdir("./Audio"):
  audio_path = './Audio/' + filename
  x , sr = librosa.load(audio_path,sr = 16000)
  print(audio_path)
  print(type(x), type(sr))
  print(x.shape, sr)
  ipd.Audio(audio_path)
  
  librosa.display.waveplot(x, sr=sr)
  mfccs = librosa.feature.mfcc(x, sr=sr)
  print(mfccs.shape)
  librosa.display.specshow(mfccs, sr=sr, x_axis='time')
  print()
  
  
#   X = librosa.stft(x)
#   Xdb = librosa.amplitude_to_db(abs(X))
#   plt.figure(figsize=(14, 5))
#   librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
#   plt.colorbar()
  break